<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/10_APIs_y_JSON.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🌐 APIs y JSON

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🟡 Intermedio  
**Duración estimada:** 60 minutos  
**Requisitos:** Haber completado el [Notebook 09 — Web Scraping Básico](09_Web_Scraping_Basico.ipynb)

---

En este notebook vas a:
- Entender qué es una API REST y cómo funciona
- Trabajar con el formato JSON (leer, escribir, transformar)
- Consumir APIs públicas con `requests`
- Manejar autenticación, parámetros y paginación
- Construir funciones reutilizables para consultar APIs
- Combinar datos de múltiples APIs con Pandas

> 💡 Las **APIs** son la forma profesional de obtener datos. Mientras que el web scraping "raspa" páginas web, las APIs te entregan datos limpios y estructurados directamente.


---

## 0. Preparación


In [ ]:
import requests
import json
import pandas as pd
import time
import os

os.makedirs("datos", exist_ok=True)

print("✅ Librerías listas")

---

## 1. ¿Qué es una API?

**API** (Application Programming Interface) es un intermediario que permite que dos programas se comuniquen. Cuando usas una app de clima, la app le pide datos al servidor a través de una API.

### Analogía: el restaurante 🍽️

| Concepto | Restaurante | API |
|----------|------------|-----|
| **Cliente** | Tú | Tu programa Python |
| **Menú** | Carta de platillos | Documentación de la API |
| **Mesero** | Quien lleva tu orden | La API (intermediario) |
| **Cocina** | Donde preparan la comida | El servidor |
| **Platillo** | Lo que recibes | Los datos (JSON) |

### API REST

La mayoría de APIs modernas son **REST** (Representational State Transfer) y funcionan sobre HTTP:

| Método HTTP | Acción | Ejemplo |
|------------|--------|---------|
| **GET** | Obtener datos | Consultar el clima |
| **POST** | Enviar/crear datos | Crear un usuario |
| **PUT** | Actualizar datos | Editar un perfil |
| **DELETE** | Eliminar datos | Borrar un registro |

En este notebook nos enfocaremos en **GET** — obtener datos.


---

## 2. JSON: el idioma de las APIs

**JSON** (JavaScript Object Notation) es el formato estándar para intercambiar datos en APIs. Se parece mucho a los diccionarios de Python:

```json
{
    "nombre": "Ver de Verdad",
    "sucursales": 144,
    "ciudades": ["Culiacán", "Mazatlán", "Los Mochis"],
    "activa": true
}
```

### 2.1 JSON en Python


In [ ]:
import json

# Diccionario de Python → JSON (string)
datos = {
    "empresa": "Ver de Verdad",
    "sucursales": 144,
    "ciudades": ["Culiacán", "Mazatlán", "Los Mochis"],
    "fundada": 2012,
    "activa": True,
    "ceo": None  # None se convierte en null en JSON
}

# Convertir a JSON (serializar)
json_string = json.dumps(datos, ensure_ascii=False, indent=2)
print("Python → JSON:")
print(json_string)
print(f"\nTipo: {type(json_string)}")  # str

In [ ]:
# JSON (string) → Diccionario de Python (deserializar)
json_texto = '{"nombre": "Hugo", "edad": 35, "ciudad": "Culiacán"}'

datos = json.loads(json_texto)
print(f"JSON → Python: {datos}")
print(f"Tipo: {type(datos)}")  # dict
print(f"Nombre: {datos['nombre']}")
print(f"Ciudad: {datos['ciudad']}")

In [ ]:
# Guardar JSON en archivo
datos_empresa = {
    "empresa": "Ver de Verdad",
    "sucursales": [
        {"nombre": "Centro", "ciudad": "Culiacán", "ventas": 185000},
        {"nombre": "Mazatlán", "ciudad": "Mazatlán", "ventas": 162000},
    ]
}

with open("datos/empresa.json", "w", encoding="utf-8") as f:
    json.dump(datos_empresa, f, ensure_ascii=False, indent=2)
print("✅ empresa.json guardado")

# Leer JSON desde archivo
with open("datos/empresa.json", "r", encoding="utf-8") as f:
    datos_leidos = json.load(f)

print(f"Empresa: {datos_leidos['empresa']}")
print(f"Sucursales: {len(datos_leidos['sucursales'])}")

### 2.2 JSON anidado (lo que devuelven las APIs)

Las respuestas reales de APIs suelen tener estructuras anidadas:


In [ ]:
# Ejemplo de respuesta típica de una API
respuesta_api = {
    "status": "success",
    "count": 2,
    "data": {
        "pais": "México",
        "estados": [
            {
                "nombre": "Sinaloa",
                "capital": "Culiacán",
                "municipios": 18,
                "poblacion": 3026943,
                "ciudades_principales": [
                    {"nombre": "Culiacán", "poblacion": 1003530},
                    {"nombre": "Mazatlán", "poblacion": 502547},
                    {"nombre": "Los Mochis", "poblacion": 416299},
                ]
            },
            {
                "nombre": "Sonora",
                "capital": "Hermosillo",
                "municipios": 72,
                "poblacion": 2944840,
            }
        ]
    }
}

# Navegar la estructura anidada
print(f"País: {respuesta_api['data']['pais']}")
print(f"Estados: {respuesta_api['count']}")

# Acceder a datos profundos
sinaloa = respuesta_api["data"]["estados"][0]
print(f"\nEstado: {sinaloa['nombre']}")
print(f"Capital: {sinaloa['capital']}")
print(f"Población: {sinaloa['poblacion']:,}")

# Ciudades principales
print(f"\nCiudades principales de {sinaloa['nombre']}:")
for ciudad in sinaloa["ciudades_principales"]:
    print(f"  📍 {ciudad['nombre']}: {ciudad['poblacion']:,} hab.")

---

## 3. Tu primera llamada a una API

Vamos a usar APIs públicas y gratuitas que no requieren autenticación.

### 3.1 API de actividades aburridas (Bored API)


In [ ]:
# API simple: actividad aleatoria
url = "https://www.boredapi.com/api/activity"
respuesta = requests.get(url)

print(f"Status: {respuesta.status_code}")
print(f"Headers Content-Type: {respuesta.headers['Content-Type']}")

# La respuesta ya viene en JSON
datos = respuesta.json()  # Equivalente a json.loads(respuesta.text)
print(f"\nRespuesta completa:")
print(json.dumps(datos, indent=2))

print(f"\n🎯 Actividad sugerida: {datos['activity']}")
print(f"   Tipo: {datos['type']}")
print(f"   Participantes: {datos['participants']}")

### 3.2 API con parámetros: universidades por país


In [ ]:
# API de universidades — acepta parámetros de búsqueda
url = "http://universities.hipolabs.com/search"
parametros = {
    "country": "Mexico",
    "name": "tecnologico"  # Buscar universidades con "tecnologico" en el nombre
}

respuesta = requests.get(url, params=parametros)
print(f"URL completa: {respuesta.url}")
print(f"Status: {respuesta.status_code}")

universidades = respuesta.json()
print(f"\nUniversidades encontradas: {len(universidades)}")

# Convertir a DataFrame
df_unis = pd.DataFrame(universidades)
print(f"\nColumnas: {list(df_unis.columns)}")
df_unis[["name", "state-province", "web_pages"]].head(10)

In [ ]:
# Todas las universidades de México
parametros = {"country": "Mexico"}
respuesta = requests.get(url, params=parametros)
unis_mexico = respuesta.json()

df_mexico = pd.DataFrame(unis_mexico)
print(f"Total universidades en México: {len(df_mexico)}")

# Análisis rápido
print(f"\nEstados con más universidades:")
if "state-province" in df_mexico.columns:
    print(df_mexico["state-province"].value_counts().head(10))

---

## 4. APIs financieras: tipo de cambio


In [ ]:
# API de tipo de cambio (gratuita, sin API key)
url = "https://open.er-api.com/v6/latest/MXN"
respuesta = requests.get(url)

if respuesta.status_code == 200:
    datos = respuesta.json()
    print(f"Base: {datos['base_code']}")
    print(f"Última actualización: {datos.get('time_last_update_utc', 'N/A')}")

    # Monedas que nos interesan
    monedas_interes = {
        "USD": "Dólar americano",
        "EUR": "Euro",
        "GBP": "Libra esterlina",
        "JPY": "Yen japonés",
        "CAD": "Dólar canadiense",
        "BRL": "Real brasileño",
        "COP": "Peso colombiano",
        "ARS": "Peso argentino",
    }

    rates = datos["rates"]
    print(f"\n💱 Tipo de cambio (1 MXN =)")
    print("-" * 45)
    for codigo, nombre in monedas_interes.items():
        if codigo in rates:
            tasa = rates[codigo]
            print(f"  {codigo} ({nombre:<20}): {tasa:.6f}")

    # ¿Cuántos pesos por 1 dólar?
    usd_rate = rates["USD"]
    mxn_por_usd = 1 / usd_rate
    print(f"\n🇺🇸 1 USD = ${mxn_por_usd:.2f} MXN")
else:
    print(f"Error: {respuesta.status_code}")

---

## 5. API del clima: Open-Meteo

[Open-Meteo](https://open-meteo.com/) es una API de clima gratuita que no requiere API key.


In [ ]:
# Clima actual de Culiacán
url = "https://api.open-meteo.com/v1/forecast"
parametros = {
    "latitude": 24.8049,
    "longitude": -107.3940,
    "current": "temperature_2m,relative_humidity_2m,wind_speed_10m,weather_code",
    "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum",
    "timezone": "America/Mazatlan",
    "forecast_days": 7,
}

respuesta = requests.get(url, params=parametros)
datos = respuesta.json()

# Clima actual
actual = datos["current"]
print("🌡️ Clima actual en Culiacán:")
print(f"  Temperatura: {actual['temperature_2m']}°C")
print(f"  Humedad: {actual['relative_humidity_2m']}%")
print(f"  Viento: {actual['wind_speed_10m']} km/h")

# Pronóstico 7 días
print(f"\n📅 Pronóstico de 7 días:")
diario = datos["daily"]
for i in range(len(diario["time"])):
    fecha = diario["time"][i]
    temp_max = diario["temperature_2m_max"][i]
    temp_min = diario["temperature_2m_min"][i]
    lluvia = diario["precipitation_sum"][i]
    lluvia_icon = "🌧️" if lluvia > 0 else "☀️"
    print(f"  {fecha}: {temp_min}° - {temp_max}°C {lluvia_icon} ({lluvia}mm)")

In [ ]:
# Comparar clima de varias ciudades de Sinaloa
ciudades = {
    "Culiacán": (24.8049, -107.3940),
    "Mazatlán": (23.2494, -106.4111),
    "Los Mochis": (25.7908, -108.9939),
    "Guasave": (25.5667, -108.4667),
}

resultados = []
for ciudad, (lat, lon) in ciudades.items():
    params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m,relative_humidity_2m,wind_speed_10m",
        "timezone": "America/Mazatlan",
    }
    resp = requests.get("https://api.open-meteo.com/v1/forecast", params=params)
    datos = resp.json()["current"]

    resultados.append({
        "ciudad": ciudad,
        "temperatura_c": datos["temperature_2m"],
        "humedad_pct": datos["relative_humidity_2m"],
        "viento_kmh": datos["wind_speed_10m"],
    })
    time.sleep(0.3)

df_clima = pd.DataFrame(resultados)
print("🌡️ Clima actual en ciudades de Sinaloa:")
df_clima

In [ ]:
# Visualizar
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

colores = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D"]

# Temperatura
axes[0].bar(df_clima["ciudad"], df_clima["temperatura_c"], color=colores)
axes[0].set_title("Temperatura (°C)", fontweight="bold")
axes[0].set_ylabel("°C")
for i, v in enumerate(df_clima["temperatura_c"]):
    axes[0].text(i, v + 0.3, f"{v}°", ha="center", fontweight="bold")

# Humedad
axes[1].bar(df_clima["ciudad"], df_clima["humedad_pct"], color=colores)
axes[1].set_title("Humedad (%)", fontweight="bold")
axes[1].set_ylabel("%")

# Viento
axes[2].bar(df_clima["ciudad"], df_clima["viento_kmh"], color=colores)
axes[2].set_title("Viento (km/h)", fontweight="bold")
axes[2].set_ylabel("km/h")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

plt.suptitle("Clima en Sinaloa — Datos en tiempo real de Open-Meteo", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 6. APIs con autenticación (API Keys)

Muchas APIs requieren una **API key** para controlar el acceso. Es como una contraseña que identifica quién hace la petición.

### ¿Cómo funciona?

```
1. Te registras en el sitio de la API
2. Te dan una API key (cadena de texto única)
3. La incluyes en tus peticiones
```

### Formas de enviar la API key:

```python
# 1. Como parámetro en la URL
requests.get("https://api.ejemplo.com/datos?api_key=TU_CLAVE")

# 2. En los headers (más seguro)
headers = {"Authorization": "Bearer TU_CLAVE"}
requests.get("https://api.ejemplo.com/datos", headers=headers)

# 3. Como header personalizado
headers = {"X-Api-Key": "TU_CLAVE"}
requests.get("https://api.ejemplo.com/datos", headers=headers)
```

> ⚠️ **NUNCA** publiques tu API key en GitHub o código público. Usa variables de entorno.


In [ ]:
# Buena práctica: usar variables de entorno para API keys
# En Google Colab puedes usar los Secrets o userdata

# Forma 1: Variable de entorno (terminal/servidor)
# export API_KEY="tu_clave_aqui"
# api_key = os.environ.get("API_KEY")

# Forma 2: Google Colab Secrets
# from google.colab import userdata
# api_key = userdata.get("MI_API_KEY")

# Forma 3: Input (para pruebas rápidas)
# api_key = input("Tu API key: ")

# Para este notebook, usaremos APIs que NO requieren key
print("✅ En este notebook usamos solo APIs públicas sin API key")
print("💡 Cuando necesites usar API keys, recuerda NUNCA publicarlas en tu código")

---

## 7. Manejo de errores en APIs

Las APIs pueden fallar por muchas razones. Siempre maneja errores:


In [ ]:
def llamar_api(url: str, params: dict = None, headers: dict = None,
               max_reintentos: int = 3, timeout: int = 10) -> dict | None:
    """
    Hace una llamada a una API con manejo robusto de errores.

    Args:
        url: URL del endpoint de la API.
        params: Parámetros de la petición.
        headers: Headers personalizados.
        max_reintentos: Intentos máximos si falla.
        timeout: Segundos máximos de espera.

    Returns:
        Diccionario con la respuesta JSON o None si falló.
    """
    for intento in range(max_reintentos):
        try:
            respuesta = requests.get(url, params=params, headers=headers, timeout=timeout)

            # Éxito
            if respuesta.status_code == 200:
                return respuesta.json()

            # Errores conocidos
            elif respuesta.status_code == 401:
                print("🔑 Error 401: API key inválida o faltante")
                return None
            elif respuesta.status_code == 403:
                print("🚫 Error 403: Acceso prohibido")
                return None
            elif respuesta.status_code == 404:
                print(f"❌ Error 404: Recurso no encontrado ({url})")
                return None
            elif respuesta.status_code == 429:
                espera = 2 ** intento  # Backoff exponencial
                print(f"⏱️ Error 429: Demasiadas peticiones. Esperando {espera}s...")
                time.sleep(espera)
            else:
                print(f"⚠️ Error {respuesta.status_code}: {respuesta.text[:100]}")
                return None

        except requests.exceptions.Timeout:
            print(f"⏱️ Timeout en intento {intento + 1}/{max_reintentos}")
            time.sleep(1)
        except requests.exceptions.ConnectionError:
            print(f"🔌 Error de conexión en intento {intento + 1}/{max_reintentos}")
            time.sleep(2)
        except json.JSONDecodeError:
            print("📄 Error: La respuesta no es JSON válido")
            return None

    print(f"❌ Falló después de {max_reintentos} intentos")
    return None

# Probar con URL válida
datos = llamar_api("https://www.boredapi.com/api/activity")
if datos:
    print(f"\n✅ Actividad: {datos['activity']}")

# Probar con URL inválida
datos = llamar_api("https://www.boredapi.com/api/no_existe")

---

## 8. API de países: RestCountries

Una API completa con información de todos los países del mundo.


In [ ]:
# Información de México
datos = llamar_api("https://restcountries.com/v3.1/name/mexico")

if datos:
    mexico = datos[0]

    print("🇲🇽 MÉXICO")
    print(f"  Nombre oficial: {mexico['name']['official']}")
    print(f"  Capital: {', '.join(mexico.get('capital', ['N/A']))}")
    print(f"  Población: {mexico['population']:,}")
    print(f"  Área: {mexico['area']:,.0f} km²")
    print(f"  Región: {mexico['region']} / {mexico['subregion']}")
    print(f"  Moneda: {list(mexico['currencies'].values())[0]['name']} ({list(mexico['currencies'].keys())[0]})")
    print(f"  Idiomas: {', '.join(mexico['languages'].values())}")
    print(f"  Zona horaria: {', '.join(mexico['timezones'])}")
    print(f"  Bandera: {mexico['flag']}")

In [ ]:
# Comparar países de América Latina
paises_latam = ["mexico", "brazil", "argentina", "colombia", "chile",
                "peru", "ecuador", "guatemala", "cuba", "bolivia"]

datos_paises = []
for pais in paises_latam:
    datos = llamar_api(f"https://restcountries.com/v3.1/name/{pais}")
    if datos:
        p = datos[0]
        moneda = list(p.get("currencies", {}).values())
        datos_paises.append({
            "pais": p["name"]["common"],
            "capital": p.get("capital", ["N/A"])[0],
            "poblacion": p["population"],
            "area_km2": p["area"],
            "region": p.get("subregion", "N/A"),
            "moneda": moneda[0]["name"] if moneda else "N/A",
        })
    time.sleep(0.3)

df_paises = pd.DataFrame(datos_paises)
df_paises = df_paises.sort_values("poblacion", ascending=False)
print(f"✅ {len(df_paises)} países cargados")
df_paises

In [ ]:
# Visualización
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Población
df_sorted = df_paises.sort_values("poblacion", ascending=True)
axes[0].barh(df_sorted["pais"], df_sorted["poblacion"], color="#2E86AB")
axes[0].set_title("Población", fontweight="bold")
axes[0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x/1e6:.0f}M"))
axes[0].spines["top"].set_visible(False)
axes[0].spines["right"].set_visible(False)

# Densidad poblacional
df_paises["densidad"] = df_paises["poblacion"] / df_paises["area_km2"]
df_dens = df_paises.sort_values("densidad", ascending=True)
axes[1].barh(df_dens["pais"], df_dens["densidad"], color="#A23B72")
axes[1].set_title("Densidad (hab/km²)", fontweight="bold")
axes[1].spines["top"].set_visible(False)
axes[1].spines["right"].set_visible(False)

plt.suptitle("Países de América Latina — Datos de RestCountries API",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 9. Construir un cliente de API reutilizable

Cuando trabajas mucho con una API, conviene crear una clase o conjunto de funciones:


In [ ]:
class ClienteClima:
    """Cliente para la API de Open-Meteo."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    CIUDADES = {
        "Culiacán": (24.8049, -107.3940),
        "Mazatlán": (23.2494, -106.4111),
        "Los Mochis": (25.7908, -108.9939),
        "Guasave": (25.5667, -108.4667),
        "CDMX": (19.4326, -99.1332),
        "Guadalajara": (20.6597, -103.3496),
        "Monterrey": (25.6866, -100.3161),
    }

    def __init__(self, timezone="America/Mazatlan"):
        self.timezone = timezone

    def obtener_actual(self, ciudad: str) -> dict | None:
        """Obtiene el clima actual de una ciudad."""
        if ciudad not in self.CIUDADES:
            print(f"❌ Ciudad no disponible: {ciudad}")
            print(f"   Disponibles: {list(self.CIUDADES.keys())}")
            return None

        lat, lon = self.CIUDADES[ciudad]
        params = {
            "latitude": lat,
            "longitude": lon,
            "current": "temperature_2m,relative_humidity_2m,wind_speed_10m,apparent_temperature",
            "timezone": self.timezone,
        }

        datos = llamar_api(self.BASE_URL, params=params)
        if datos:
            actual = datos["current"]
            return {
                "ciudad": ciudad,
                "temperatura": actual["temperature_2m"],
                "sensacion_termica": actual["apparent_temperature"],
                "humedad": actual["relative_humidity_2m"],
                "viento": actual["wind_speed_10m"],
            }
        return None

    def obtener_pronostico(self, ciudad: str, dias: int = 7) -> pd.DataFrame | None:
        """Obtiene el pronóstico de varios días."""
        if ciudad not in self.CIUDADES:
            return None

        lat, lon = self.CIUDADES[ciudad]
        params = {
            "latitude": lat,
            "longitude": lon,
            "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max",
            "timezone": self.timezone,
            "forecast_days": dias,
        }

        datos = llamar_api(self.BASE_URL, params=params)
        if datos:
            diario = datos["daily"]
            df = pd.DataFrame({
                "fecha": diario["time"],
                "temp_max": diario["temperature_2m_max"],
                "temp_min": diario["temperature_2m_min"],
                "lluvia_mm": diario["precipitation_sum"],
                "viento_max": diario["wind_speed_10m_max"],
            })
            df["ciudad"] = ciudad
            return df
        return None

    def comparar_ciudades(self, ciudades: list = None) -> pd.DataFrame:
        """Compara el clima actual de varias ciudades."""
        if ciudades is None:
            ciudades = list(self.CIUDADES.keys())

        resultados = []
        for ciudad in ciudades:
            datos = self.obtener_actual(ciudad)
            if datos:
                resultados.append(datos)
            time.sleep(0.3)

        return pd.DataFrame(resultados)

# Usar el cliente
clima = ClienteClima()

# Clima actual
print("🌡️ Clima actual en Culiacán:")
actual = clima.obtener_actual("Culiacán")
if actual:
    for k, v in actual.items():
        print(f"  {k}: {v}")

In [ ]:
# Pronóstico de 7 días
df_pronostico = clima.obtener_pronostico("Culiacán")
if df_pronostico is not None:
    print("📅 Pronóstico 7 días — Culiacán:")
    print(df_pronostico.to_string(index=False))

In [ ]:
# Comparar todas las ciudades
df_comparar = clima.comparar_ciudades(["Culiacán", "Mazatlán", "Los Mochis", "CDMX", "Monterrey"])
print("🌡️ Comparativa de clima:")
df_comparar.sort_values("temperatura", ascending=False)

---

## 10. 🏆 Mini Proyecto: Dashboard de datos combinados

Vamos a combinar datos de múltiples APIs en un solo análisis:


In [ ]:
# 🏆 Mini Proyecto: Dashboard Multi-API

import matplotlib.pyplot as plt

print("📥 Recopilando datos de múltiples APIs...\n")

# 1. Clima de ciudades de Sinaloa
clima = ClienteClima()
df_clima = clima.comparar_ciudades(["Culiacán", "Mazatlán", "Los Mochis", "Guasave"])
print(f"✅ Clima: {len(df_clima)} ciudades")

# 2. Tipo de cambio
tc_datos = llamar_api("https://open.er-api.com/v6/latest/MXN")
monedas = {}
if tc_datos:
    for mon in ["USD", "EUR", "GBP", "CAD"]:
        monedas[mon] = 1 / tc_datos["rates"][mon]
    print(f"✅ Tipo de cambio: {len(monedas)} monedas")

# 3. Datos de México
pais_datos = llamar_api("https://restcountries.com/v3.1/name/mexico")
if pais_datos:
    poblacion_mx = pais_datos[0]["population"]
    print(f"✅ Datos de México: población {poblacion_mx:,}")

# --- Generar Dashboard ---
fig = plt.figure(figsize=(16, 10))
fig.patch.set_facecolor("#FAFAFA")
fig.suptitle("DASHBOARD MULTI-API\nDatos en tiempo real de Sinaloa",
             fontsize=18, fontweight="bold", y=0.98)

# 1. Clima
ax1 = fig.add_subplot(2, 2, 1)
if not df_clima.empty:
    colores = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D"]
    bars = ax1.bar(df_clima["ciudad"], df_clima["temperatura"], color=colores[:len(df_clima)])
    for bar, temp in zip(bars, df_clima["temperatura"]):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
                f"{temp}°C", ha="center", fontweight="bold")
    ax1.set_title("🌡️ Temperatura Actual", fontweight="bold")
    ax1.set_ylabel("°C")
    ax1.spines["top"].set_visible(False)
    ax1.spines["right"].set_visible(False)

# 2. Humedad
ax2 = fig.add_subplot(2, 2, 2)
if not df_clima.empty:
    bars = ax2.bar(df_clima["ciudad"], df_clima["humedad"], color=colores[:len(df_clima)])
    for bar, hum in zip(bars, df_clima["humedad"]):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f"{hum}%", ha="center", fontweight="bold")
    ax2.set_title("💧 Humedad Actual", fontweight="bold")
    ax2.set_ylabel("%")
    ax2.spines["top"].set_visible(False)
    ax2.spines["right"].set_visible(False)

# 3. Tipo de cambio
ax3 = fig.add_subplot(2, 2, 3)
if monedas:
    mons = list(monedas.keys())
    vals = list(monedas.values())
    bars = ax3.bar(mons, vals, color=["#2E86AB", "#A23B72", "#F18F01", "#C73E1D"])
    for bar, val in zip(bars, vals):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f"${val:.2f}", ha="center", fontweight="bold", fontsize=10)
    ax3.set_title("💱 Tipo de Cambio (1 moneda = X MXN)", fontweight="bold")
    ax3.set_ylabel("Pesos mexicanos")
    ax3.spines["top"].set_visible(False)
    ax3.spines["right"].set_visible(False)

# 4. Info box
ax4 = fig.add_subplot(2, 2, 4)
ax4.axis("off")
info = [
    "📊 Fuentes de datos:",
    "",
    "🌡️ Open-Meteo API (clima)",
    "💱 Exchange Rate API (tipo de cambio)",
    "🌎 RestCountries API (países)",
    "",
    f"🇲🇽 Población de México: {poblacion_mx:,}" if pais_datos else "",
    f"💵 1 USD = ${monedas.get('USD', 0):.2f} MXN" if monedas else "",
    "",
    "Generado con Python | Culiacan.AI",
]
ax4.text(0.1, 0.9, "\n".join(info), transform=ax4.transAxes,
         fontsize=12, verticalalignment="top", fontfamily="monospace",
         bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))

plt.tight_layout(rect=[0, 0, 1, 0.95])
fig.savefig("datos/dashboard_multiapi.png", dpi=200, bbox_inches="tight", facecolor="#FAFAFA")
print("\n✅ Dashboard guardado como datos/dashboard_multiapi.png")
plt.show()

---

## 🔥 Retos

1. **Monitor de tipo de cambio:** Crea una función que consulte el tipo de cambio USD/MXN cada cierto tiempo (simula con un loop de 5 iteraciones con pausa de 2 segundos). Guarda los resultados en un DataFrame con timestamp y grafica la "tendencia".

2. **Buscador de universidades:** Crea un programa interactivo donde el usuario ingrese un país y una palabra clave, y el programa busque universidades usando la API de hipolabs. Muestra los resultados en una tabla bonita.

3. **Comparador de clima internacional:** Usando Open-Meteo, compara el pronóstico de 7 días de Culiacán con otra ciudad del mundo (Tokyo, Londres, Nueva York). Genera una gráfica con las temperaturas máximas y mínimas de ambas ciudades superpuestas.


In [ ]:
# Reto 1: Monitor de tipo de cambio
# Tu código aquí 👇


In [ ]:
# Reto 2: Buscador de universidades
# Tu código aquí 👇


In [ ]:
# Reto 3: Comparador de clima internacional
# Tu código aquí 👇


---

## 📋 Resumen

### JSON
| Operación | Código |
|-----------|--------|
| Dict → JSON string | `json.dumps(datos, indent=2)` |
| JSON string → Dict | `json.loads(texto)` |
| Guardar en archivo | `json.dump(datos, archivo)` |
| Leer de archivo | `json.load(archivo)` |

### Llamadas a APIs
| Operación | Código |
|-----------|--------|
| GET simple | `requests.get(url)` |
| Con parámetros | `requests.get(url, params={...})` |
| Con headers | `requests.get(url, headers={...})` |
| Obtener JSON | `respuesta.json()` |
| Status code | `respuesta.status_code` |

### Buenas prácticas
| Práctica | Por qué |
|----------|---------|
| Manejar errores | Las APIs fallan |
| Pausas entre llamadas | No sobrecargar servidores |
| Guardar API keys seguras | Nunca en código público |
| Cachear respuestas | No repetir llamadas innecesarias |
| Leer la documentación | Cada API tiene sus reglas |

### APIs públicas útiles
| API | Qué ofrece | URL |
|-----|-----------|-----|
| Open-Meteo | Clima | open-meteo.com |
| ExchangeRate | Tipo de cambio | open.er-api.com |
| RestCountries | Datos de países | restcountries.com |
| Universities | Universidades | hipolabs.com |
| JSONPlaceholder | Datos de prueba | jsonplaceholder.typicode.com |

---

## ⏭️ ¿Qué sigue?

¡Felicidades! 🎉 Completaste el **Bloque 2: Python Práctico**. Ya puedes leer archivos, analizar datos, crear gráficas, hacer scraping y consumir APIs.

En el siguiente notebook empezamos el **Bloque 3: Python + IA** con **Intro a NumPy** — la librería fundamental para computación numérica y machine learning.

👉 [11 — Intro a NumPy](11_Intro_a_NumPy.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
